# Part One

In [84]:
# Import Libraries
import pandas as pd
import numpy as np
import yaml
import matplotlib.pyplot as plt
import seaborn as sns
#Libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from matplotlib.pyplot import figure
import pyodbc 
import glob

In [13]:
# Localise with . files 
# config_filename = 'C:/Users/cra/OneDrive - San Diego Association of Governments/DS41_42/ds41_42_config.yml'
config_filename = './ds41_42_config.yml'

In [14]:
config_filename

'./ds41_42_config.yml'

In [15]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [36]:
first_ID = config['inputs']['first_ID']

In [37]:
second_ID = config['inputs']['second_ID']

In [38]:
first_ID_datafiles = config[first_ID].values()

In [39]:
second_ID_datafiles = config[second_ID].values()

In [40]:
# If collecting user input from Jupyter Notebook
# choice1 = input("Please input your first choice: ")
# choice1

## Downloading SQL Data

In [55]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=DDAMWSQL16.sandag.org;'
                      'Database=demographic_warehouse;'
                      'Trusted_Connection=yes;')

In [56]:
query_all = "SELECT mgra, geotype, geozone FROM demographic_warehouse.dim.mgra WHERE series = 14 AND (geotype='cpa' OR geotype='jurisdiction')" #Remove the last and part when I do this for real 

In [57]:
sql_query = pd.read_sql_query(query_all,conn)

sql_df_all = pd.DataFrame(sql_query)

In [58]:
# SQl Data at different levels 
jur_level = sql_df_all[sql_df_all['geotype']=='jurisdiction']
cpa_level = sql_df_all[sql_df_all['geotype']=='cpa']

## Downloading the first_ID Data

In [128]:
# first_ID Data Frame
first_ID_df = pd.DataFrame()
for file_name in first_ID_datafiles:
    working_df = pd.read_csv(file_name)
    working_df['year'] = f"{file_name[-11:-7]}"
    first_ID_df = first_ID_df.append(working_df)

In [129]:
first_ID_df

,mgra,taz,hs,hs_sf,hs_mf,hs_mh,hh,hh_sf,hh_mf,hh_mh,...,upscaleroom,hotelroomtotal,luz_id,truckregiontype,district27,milestocoast,acres,effective_acres,land_acres,year
0,1,3331,19,19,0,0,18,18,0,0,...,0,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016
1,2,3331,35,35,0,0,34,34,0,0,...,0,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016
2,3,3358,52,52,0,0,52,52,0,0,...,0,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016
3,4,3358,30,30,0,0,30,30,0,0,...,0,0,95,1,27,4.2782,7.976178,7.976178,7.976178,2016
4,5,3358,28,28,0,0,28,28,0,0,...,0,0,95,1,27,4.0062,7.072502,7.063693,7.072502,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22997,22998,1290,92,92,0,0,87,87,0,0,...,0,0,14,1,1,2.6193,41.241522,26.863578,41.241522,2050
22998,22999,1290,0,0,0,0,0,0,0,0,...,0,0,14,1,1,2.3703,35.842780,30.635095,35.842780,2050
22999,23000,1290,131,131,0,0,126,126,0,0,...,0,0,14,1,1,2.1721,28.735275,16.109816,28.735275,2050
23000,23001,1254,85,85,0,0,82,82,0,0,...,0,0,14,1,1,2.7063,41.006144,28.126056,41.006144,2050


In [133]:
# Save the features for future use
features = first_ID_df.drop(['mgra', 'year'], axis=1).columns

In [134]:
#Making it original
first_ID_df.columns = [column_name + f'_{first_ID}' for column_name in first_ID_df.columns]

In [135]:
import copy 
comparison_first_ID_no_geozone = copy.deepcopy(first_ID_df)

In [136]:
# Adding SQl Data to first_id_df
comparison_first_ID_processed_data = comparison_first_ID_no_geozone.merge(jur_level, how = 'left', left_on = f'mgra_{first_ID}', right_on = 'mgra')

In [137]:
# Drop duplicate mgra column
comparison_first_ID_processed_data = comparison_first_ID_processed_data.drop('mgra', axis=1)

In [138]:
comparison_first_ID_processed_data.head()

,mgra_DS35,taz_DS35,hs_DS35,hs_sf_DS35,hs_mf_DS35,hs_mh_DS35,hh_DS35,hh_sf_DS35,hh_mf_DS35,hh_mh_DS35,...,luz_id_DS35,truckregiontype_DS35,district27_DS35,milestocoast_DS35,acres_DS35,effective_acres_DS35,land_acres_DS35,year_DS35,geotype,geozone
0,1,3331,19,19,0,0,18,18,0,0,...,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,jurisdiction,San Diego
1,1,3331,19,19,0,0,18,18,0,0,...,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,jurisdiction,San Diego
2,2,3331,35,35,0,0,34,34,0,0,...,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,jurisdiction,San Diego
3,2,3331,35,35,0,0,34,34,0,0,...,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,jurisdiction,San Diego
4,3,3358,52,52,0,0,52,52,0,0,...,95,1,27,4.1939,27.845124,26.867938,27.845124,2016,jurisdiction,San Diego


In [54]:
# first_ID_df.to_csv(f"/comparison_{first_ID}_processed_data")

## Downloading  the second_ID Data

In [76]:
# second_id Data Frame
second_ID_df = pd.DataFrame()
for file_name in second_ID_datafiles:
    working_df = pd.read_csv(file_name)
    working_df['year'] = f"{file_name[-11:-7]}"
    second_ID_df = second_ID_df.append(working_df)

In [77]:
# Specify df in column names
second_ID_df.columns = [x + f'_{second_ID}' for x in second_ID_df.columns]

In [113]:
second_ID_df

,mgra_DS41,taz_DS41,hs_DS41,hs_sf_DS41,hs_mf_DS41,hs_mh_DS41,hh_DS41,hh_sf_DS41,hh_mf_DS41,hh_mh_DS41,...,upscaleroom_DS41,hotelroomtotal_DS41,luz_id_DS41,truckregiontype_DS41,district27_DS41,milestocoast_DS41,acres_DS41,effective_acres_DS41,land_acres_DS41,year_DS41
0,1,3331,19,19,0,0,18,18,0,0,...,0,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016
1,2,3331,35,35,0,0,34,34,0,0,...,0,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016
2,3,3358,52,52,0,0,52,52,0,0,...,0,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016
3,4,3358,30,30,0,0,30,30,0,0,...,0,0,95,1,27,4.2782,7.976178,7.976178,7.976178,2016
4,5,3358,28,28,0,0,28,28,0,0,...,0,0,95,1,27,4.0062,7.072502,7.063693,7.072502,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22997,22998,1290,92,92,0,0,87,87,0,0,...,0,0,14,1,1,2.6193,41.241522,26.863578,41.241522,2050
22998,22999,1290,0,0,0,0,0,0,0,0,...,0,0,14,1,1,2.3703,35.842780,30.635095,35.842780,2050
22999,23000,1290,131,131,0,0,126,126,0,0,...,0,0,14,1,1,2.1721,28.735275,16.109816,28.735275,2050
23000,23001,1254,85,85,0,0,81,81,0,0,...,0,0,14,1,1,2.7063,41.006144,28.126056,41.006144,2050


In [79]:
comparison_second_ID_no_geozone = copy.deepcopy(second_ID_df)

In [93]:
# Adding SQl Data to second_id_df
comparison_second_ID_processed_data = comparison_second_ID_no_geozone.merge(jur_level, how = 'left', left_on = f'mgra_{second_ID}', right_on = 'mgra')

In [94]:
# Drop duplicate mgra column
comparison_second_ID_processed_data = comparison_second_ID_processed_data.drop('mgra', axis=1)

In [95]:
comparison_second_ID_processed_data.head()

,mgra_DS41,taz_DS41,hs_DS41,hs_sf_DS41,hs_mf_DS41,hs_mh_DS41,hh_DS41,hh_sf_DS41,hh_mf_DS41,hh_mh_DS41,...,luz_id_DS41,truckregiontype_DS41,district27_DS41,milestocoast_DS41,acres_DS41,effective_acres_DS41,land_acres_DS41,year_DS41,geotype,geozone
0,1,3331,19,19,0,0,18,18,0,0,...,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,jurisdiction,San Diego
1,1,3331,19,19,0,0,18,18,0,0,...,95,1,27,3.7997,16.615444,12.961482,16.615444,2016,jurisdiction,San Diego
2,2,3331,35,35,0,0,34,34,0,0,...,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,jurisdiction,San Diego
3,2,3331,35,35,0,0,34,34,0,0,...,95,1,27,3.9761,19.519185,19.519185,19.519185,2016,jurisdiction,San Diego
4,3,3358,52,52,0,0,52,52,0,0,...,95,1,27,4.1939,27.845124,26.867938,27.845124,2016,jurisdiction,San Diego


In [82]:
# second_ID_df.to_csv(f"/comparison_{second_ID}_processed_data")

## Concatenate both DS dataframes

In [115]:
# Inner join dataframes on mgra and year
first_second_ID_comparison = first_ID_df.merge(second_ID_df, left_on=[f"mgra_{first_ID}", f"year_{first_ID}"], right_on=[f"mgra_{second_ID}", f"year_{second_ID}"])

In [119]:
first_second_ID_comparison.head()

,mgra_DS35,taz_DS35,hs_DS35,hs_sf_DS35,hs_mf_DS35,hs_mh_DS35,hh_DS35,hh_sf_DS35,hh_mf_DS35,hh_mh_DS35,...,upscaleroom_DS41,hotelroomtotal_DS41,luz_id_DS41,truckregiontype_DS41,district27_DS41,milestocoast_DS41,acres_DS41,effective_acres_DS41,land_acres_DS41,year_DS41
0,1,3331,19,19,0,0,18,18,0,0,...,0,0,95,1,27,3.7997,16.615444,12.961482,16.615444,2016
1,2,3331,35,35,0,0,34,34,0,0,...,0,0,95,1,27,3.9761,19.519185,19.519185,19.519185,2016
2,3,3358,52,52,0,0,52,52,0,0,...,0,0,95,1,27,4.1939,27.845124,26.867938,27.845124,2016
3,4,3358,30,30,0,0,30,30,0,0,...,0,0,95,1,27,4.2782,7.976178,7.976178,7.976178,2016
4,5,3358,28,28,0,0,28,28,0,0,...,0,0,95,1,27,4.0062,7.072502,7.063693,7.072502,2016


In [121]:
# Clean up combined comparison df
first_second_ID_comparison = first_second_ID_comparison.rename(columns={f'mgra_{first_ID}':'mgra', f'year_{first_ID}': 'year'})
first_second_ID_comparison = first_second_ID_comparison.drop([f'mgra_{second_ID}', f'year_{second_ID}'], axis=1)

In [126]:
# Aggregate sum by mgra and year values
first_second_ID_comparison = first_second_ID_comparison.groupby(['mgra', 'year']).sum()

In [127]:
first_second_ID_comparison

taz_DS35  hs_DS35  hs_sf_DS35  hs_mf_DS35  hs_mh_DS35  hh_DS35  \
mgra  year                                                                   
1     2016      3331       19          19           0           0       18   
      2018      3331       19          19           0           0       18   
      2020      3331       19          19           0           0       18   
      2023      3331       20          20           0           0       18   
      2025      3331       20          20           0           0       18   
...              ...      ...         ...         ...         ...      ...   
23002 2032      1254      120          20         100           0      100   
      2035      1254      120          20         100           0      100   
      2040      1254      120          20         100           0      106   
      2045      1254      120          20         100           0      107   
      2050      1254      120          20         100           0      109   

            hh_sf_DS35  hh_mf_DS35  hh_mh_DS35  gq_civ_DS35  ...  \
mgra  year                                                   ...   
1     2016          18           0           0            0  ...   
      2018          18           0           0            0  ...   
      2020          18           0           0            0  ...   
      2023          18           0           0            0  ...   
      2025          18           0           0            0  ...   
...                ...         ...         ...          ...  ...   
23002 2032          14          86           0            0  ...   
      2035          14          86           0            0  ...   
      2040          16          90           0            0  ...   
      2045          17          90           0            0  ...   
      2050          17          92           0            0  ...   

            midpriceroom_DS41  upscaleroom_DS41  hotelroomtotal_DS41  \
mgra  year                                                             
1     2016                  0                 0                    0   
      2018                  0                 0                    0   
      2020                  0                 0                    0   
      2023                  0                 0                    0   
      2025                  0                 0                    0   
...                       ...               ...                  ...   
23002 2032                  0                 0                    0   
      2035                  0                 0                    0   
      2040                  0                 0                    0   
      2045                  0                 0                    0   
      2050                  0                 0                    0   

            luz_id_DS41  truckregiontype_DS41  district27_DS41  \
mgra  year                                                       
1     2016           95                     1               27   
      2018           95                     1               27   
      2020           95                     1               27   
      2023           95                     1               27   
      2025           95                     1               27   
...                 ...                   ...              ...   
23002 2032           14                     1                1   
      2035           14                     1                1   
      2040           14                     1                1   
      2045           14                     1                1   
      2050           14                     1                1   

            milestocoast_DS41  acres_DS41  effective_acres_DS41  \
mgra  year                                                        
1     2016             3.7997   16.615444             12.961482   
      2018             3.7997   16.615444             12.961482   
      2020             3.7997   16.615444       

In [142]:
# Calculate diff values between the two DS_ID's
diff_df = pd.DataFrame()
for column in features:
    #diff_df[f'{column}_diff'] = first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}']
    diff_df[f'{column}_diff'] = abs(first_second_ID_comparison[f'{column}_{first_ID}'] - first_second_ID_comparison[f'{column}_{second_ID}'])

In [143]:
diff_df

taz_diff  hs_diff  hs_sf_diff  hs_mf_diff  hs_mh_diff  hh_diff  \
mgra  year                                                                   
1     2016         0        0           0           0           0        0   
      2018         0        0           0           0           0        0   
      2020         0        0           0           0           0        0   
      2023         0        0           0           0           0        0   
      2025         0        0           0           0           0        0   
...              ...      ...         ...         ...         ...      ...   
23002 2032         0        0           0           0           0        2   
      2035         0        0           0           0           0        2   
      2040         0        0           0           0           0        4   
      2045         0        0           0           0           0        2   
      2050         0        0           0           0           0        1   

            hh_sf_diff  hh_mf_diff  hh_mh_diff  gq_civ_diff  ...  \
mgra  year                                                   ...   
1     2016           0           0           0            0  ...   
      2018           0           0           0            0  ...   
      2020           0           0           0            0  ...   
      2023           0           0           0            0  ...   
      2025           0           0           0            0  ...   
...                ...         ...         ...          ...  ...   
23002 2032           3           5           0            0  ...   
      2035           3           5           0            0  ...   
      2040           2           6           0            0  ...   
      2045           3           1           0            0  ...   
      2050           3           2           0            0  ...   

            midpriceroom_diff  upscaleroom_diff  hotelroomtotal_diff  \
mgra  year                                                             
1     2016                  0                 0                    0   
      2018                  0                 0                    0   
      2020                  0                 0                    0   
      2023                  0                 0                    0   
      2025                  0                 0                    0   
...                       ...               ...                  ...   
23002 2032                  0                 0                    0   
      2035                  0                 0                    0   
      2040                  0                 0                    0   
      2045                  0                 0                    0   
      2050                  0                 0                    0   

            luz_id_diff  truckregiontype_diff  district27_diff  \
mgra  year                                                       
1     2016            0                     0                0   
      2018            0                     0                0   
      2020            0                     0                0   
      2023            0                     0                0   
      2025            0                     0                0   
...                 ...                   ...              ...   
23002 2032            0                     0                0   
      2035            0                     0                0   
      2040            0                     0                0   
      2045            0                     0                0   
      2050            0                     0                0   

            milestocoast_diff  acres_diff  effective_acres_diff  \
mgra  year                                                        
1     2016                0.0         0.0                   0.0   
      2018                0.0         0.0                   0.0   
      2020                0.0         0.0       